Able to take in an array representation of a scrabble board and outputs the possible moves and their scores.

Method:

1. Finds all possible placements of hand letters on a per-row basis

2. Checks to see if these words are in the dictionary

3. Finds the crossing words the valid words would make

4. Checks to see if all crossing words are valid per valid main word

5. Scores these words by adding up their tile values, after letter multipliers are applied. Then word multipliers are applied. This is through their interaction with multiplier arrays, which have been masked for tiles previously placed on the board.

6. Board is transposed and 1-5 are run again for the column words.

Possible improvements:
1. Biggest one is improving the time. I think I should be able to check all the crossing words at the same time rather than for each individual main word. It takes around 40 seconds now.
    * Actually, at this point, all the time seems to come from generating the permutations
    * Almost all the time comes in one for loop that generates the permutations
2. Edge words and crossing words should be able to use the same method. You would just have more hook letters and no hand letters for crossing words. Would that be worth it though? Probably not.

3. The dictionary I use is slightly different than the official WWF dictionary, so there will be some differences.

4. Stop transposing the original board, and just refer to the transposed board when you need it.

5. Another strategy could be to pre-separate the dictionary according to word length, then use regex to find which of those words would fit on the board.

6. Figure out the "leave values" for hand tiles and add them to the board scores to find the expected value of a play.

7. Use this for learning moves https://www.cross-tables.com/annolistself.php , (Or perhaps from the scrabblr github)

8. Use a more sophisticated version of leave values from here: https://www.cross-tables.com/leaves.php

In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import csv
import itertools
import re
from collections import defaultdict
import time

In [19]:
class Scrabbler:
    # https://raw.githubusercontent.com/jmlewis/valett/master/scrabble/sowpods.txt
    data_path = "../sowpods.txt"  # from above
    with open(data_path, newline="") as f:
        reader = csv.reader(f)
        allwords = list(reader)
    WORDLIST = list(itertools.chain.from_iterable(allwords))
    WORDLIST = [x.lower() for x in WORDLIST]
    WORDSET = set(WORDLIST)
    BOARD_LENGTH = 15
    MAX_I = 14
    row_mainword_scores = {}
    column_mainword_scores = {}

    LETTER_MULTIPLIERS = [
        [1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1],  # 0
        [1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1],
        [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1],  # 2
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],
        [1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1],  # 4
        [1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1],
        [3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3],  # 6
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3],  # 8
        [1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1],
        [1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1],  # 10
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],
        [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1],  # 12
        [1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1],
        [1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1],  # 14
    ]
    LETTER_MULTIPLIERS_ARRAY = np.array(LETTER_MULTIPLIERS)

    WORD_MULTIPLIERS = [
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],  # 0
        [1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 2
        [3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 4
        [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 6
        [1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 8
        [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 10
        [3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 12
        [1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1],
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],  # 14
    ]
    WORD_MULTIPLIERS_ARRAY = np.array(WORD_MULTIPLIERS)

    TILE_SCORES = {
        "a": 1,
        "b": 4,
        "c": 4,
        "d": 2,
        "e": 1,
        "f": 4,
        "g": 3,
        "h": 3,
        "i": 1,
        "j": 10,
        "k": 5,
        "l": 2,
        "m": 4,
        "n": 2,
        "o": 1,
        "p": 4,
        "q": 10,
        "r": 1,
        "s": 1,
        "t": 1,
        "u": 2,
        "v": 5,
        "w": 4,
        "x": 8,
        "y": 3,
        "z": 10,
    }
    
    #The leave values for the hand tiles. Basically, the present value of tile in the hand
    #Add the the total value of the tiles in the hand to the play on the board to find the 
    #expected value of a play
    LEAVE_VALUES = {
        "a": 1.7,
        "b": -3.5,
        "c": -2.0,
        "d": 0.0,
        "e": 2.7,
        "f": -2.5,
        "g": -4.5,
        "h": 2.0,
        "i": -0.3,
        "j": -4.5,
        "k": -0.5,
        "l": -1.0,
        "m": -1.0,
        "n": -1.0,
        "o": 0.2,
        "p": -2.5,
        "q": -9.5,
        "r": 0.5,
        "s": 7.5,
        "t": -0.5,
        "u": -4.3,
        "v": -7.5,
        "w": -3.5,
        "x": 2.0,
        "y": 0.0,
        "z": 4.5,
    }

    def __init__(self, board, hand_letters):
        self.board = board
        self.hand_letters = hand_letters
        self.T_board = np.copy(board).transpose()  # TODO: should this be here?
        self.board_mask = (np.copy(board) != "").astype(int)

    # Navigator functions
    def check_yx(self, y, x):
        if x > -1 and x < 15 and y > -1 and y < 15:
            return bool(self.board[y, x])
        else:
            return "out"

    def check_right(self, y, x):
        if x < self.BOARD_LENGTH - 1:
            return bool(self.board[y, x + 1]), self.board[y, x + 1]
        else:
            return "out"

    def check_left(self, y, x):
        if x > 0:
            return bool(self.board[y, x - 1]), self.board[y, x - 1]
        else:
            return "out"

    def check_above(self, y, x):
        if y > 0:
            return bool(self.board[y - 1, x]), self.board[y - 1, x]
        else:
            return "out"

    def check_below(self, y, x):
        if y < self.BOARD_LENGTH - 1:
            return bool(self.board[y + 1, x]), self.board[y + 1, x]
        else:
            return "out"

    # For potential words of a given length
    # start at the left, travel right (keep track of empties) till you hit a letter
    def hand_letter_permutations(self, length):
        # this returns a list of permutations of the hand letters of a given length
        letter_permutations = list(itertools.permutations(self.hand_letters, length))
        return letter_permutations

    # Find all the possible ways of placing tiles on the board in a row
    # TODO: how to speed up this? It takes around 40 seconds in total, almost all the time of the program
    def row_permutations(self, row):
        y = row
        board_row = self.board[row]

        # no_ means "number of" here
        no_hand_letters = len(self.hand_letters)
        no_pre_filled = sum(list(map(bool, board_row)))
        row_perms_dict = {}

        if no_pre_filled > 0:
            for perm_len in range(1, no_hand_letters + 1):
                hand_perms = self.hand_letter_permutations(perm_len)
                # for x in range(0, BOARD_LENGTH-(len(self.hand_letters)-1)-no_pre_filled):
                # x=0
                no_tiles_to_right = no_pre_filled
                for hand_perm in hand_perms:
                    # while x + no_tiles_to_right+perm_len<self.BOARD_LENGTH:
                    x = 0
                    if (
                        self.check_yx(y, x) == False
                    ):  # and sum(list(map(bool,board_row[x:x+no_hand_letters])))
                        while True:
                            # TODO check this
                            if (
                                any(
                                    board_row[max(0, x - 1) : min(x + perm_len + 1, 14)]
                                )
                                == True
                            ):
                                complete_word, word_y, word_x = self.complete_word_y_x(
                                    row, x, hand_perm
                                )
                                if complete_word == []:
                                    break
                                elif len(complete_word) > perm_len:
                                    # TODO: could leave out the part after the and below and delete duplicates later
                                    if (
                                        complete_word in row_perms_dict
                                        and [word_y, word_x]
                                        not in row_perms_dict[complete_word]
                                    ):
                                        row_perms_dict[complete_word].append(
                                            [word_y, word_x]
                                        )
                                    else:
                                        row_perms_dict[complete_word] = [
                                            [word_y, word_x]
                                        ]
                            x = x + 1
                            if x == self.BOARD_LENGTH:
                                break
                    # no_tiles_to_right=sum(list(map(bool,board_row[x+1:])))#TODO: +1 here?
        return row_perms_dict

    # fill the next len(hand_perm) spaces with hand tiles and find the connecting words
    # row=y
    def complete_word_y_x(self, row, x, hand_perm):
        hand_perm = list(hand_perm)
        perm_len = len(hand_perm)
        X = x
        board_row = self.board[row]
        y = row
        no_filled = 0
        filled_tiles = []
        left_letters = []
        right_letters = []
        while self.check_left(y, x)[0] == True:
            x = x - 1
            left_letters = left_letters + [board_row[x]]
        left_letters = left_letters[::-1]
        x = X
        while no_filled < perm_len:
            if self.check_yx(y, x) == True:
                filled_tiles = filled_tiles + [board_row[x]]
            elif self.check_yx(y, x) == "out":
                return [], [], []
            elif self.check_yx(y, x) == False:
                new_tile = hand_perm.pop(0)
                filled_tiles = filled_tiles + [new_tile]
                no_filled = no_filled + 1
            x = x + 1
        x = x - 1
        while self.check_right(y, x)[0] == True:
            x = x + 1
            right_letters = right_letters + [board_row[x]]
        complete_word_as_list = left_letters + filled_tiles + right_letters
        word_x = X - len(left_letters)
        complete_word = "".join(complete_word_as_list)
        return complete_word, y, word_x

    # Find the crossing words for a given complete_word_y_x info
    def find_crosses(self, complete_word, y, x):
        crossing_words = set()
        count = 0
        for tile in complete_word:
            # TODO: don't think I need this if part, only the last part and len(crossing_word)>1
            if (
                self.check_above(y, x)[0] == True or self.check_below(y, x)[0] == True
            ) and self.board[y, x] != tile:
                crossing_word, T_y, T_x = self.crossing_words_at_tile(tile, y, x)
                if len(crossing_word) > 1:
                    crossing_words.add((crossing_word, T_y, T_x))
            x = x + 1
        # TODO should I use/fix this?
        #         if count == 1 and len(crossing_words) == 0:
        #             crossing_words.add((edge_letter, edge_y, edge_x))
        if len(crossing_words) == 0:
            crossing_words.add(("", -1, -1))
        return crossing_words

    # Enter a tile and its location and return the crosses that it makes
    def crossing_words_at_tile(self, tile, y, x):
        # We will be using the transpose board so y and x are switched
        self.board = self.board.transpose()
        y, x = x, y
        crossing_word, T_y, T_x = self.complete_word_y_x(y, x, (tile))
        self.board = self.board.transpose()
        return crossing_word, T_y, T_x

    # insert the "hook letter" into a shadow board and find the output connected to that
    # assume the board has already been transposed and you are operating with the right y and x
    #TODO: probably better to create a copy and remove from that
    def edge_perms(self, hook_letter, y, x):
        self.board[y, x] = hook_letter
        self.hand_letters.remove(hook_letter)
        shadow_row_perms = self.row_permutations(y)
        edge_perms_dict = {
            k: v
            for (k, v) in shadow_row_perms.items()
            if v[0][1] <= x
            and v[0][1] + len(k)
            >= x  # TODO: see if I really need to put out a nested list here
        }

        self.hand_letters.append(hook_letter)
        self.board[y, x] = ""
        return edge_perms_dict

    # Figure out which permutations are in the dictionary
    def find_valid_perms(self):
        crossing_set = set()
        board_perms_dict = {}

        for row in range(0, self.BOARD_LENGTH):
            #Debugging
            #start = time.time()
            row_perms = self.row_permutations(row)
            #Debugging
            #print(time.time() - start)
            for complete_word in row_perms:
                if complete_word in board_perms_dict:
                    board_perms_dict[complete_word] = (
                        board_perms_dict[complete_word] + row_perms[complete_word]
                    )
                else:
                    board_perms_dict[complete_word] = row_perms[complete_word]
        perms_set = set(board_perms_dict.keys())
        valid_perms = self.WORDSET.intersection(perms_set)
        valid_perms_dict = {
            k: v for (k, v) in board_perms_dict.items() if k in valid_perms
        }
        return valid_perms_dict

    # use the perms to find crossings and check if they are all valid
    # TODO: ask to see if I can somehow check all of these sets at once so I only have to iter through once
    def check_crosses(self, valid_perm, y, x):
        crosses_with_position = self.find_crosses(valid_perm, y, x)
        crossing_words = set([i for i in zip(*crosses_with_position)][0])
        crosses_valid = crossing_words.issubset(
            self.WORDSET
        ) or crosses_with_position == {("", -1, -1)}
        return crosses_valid, crosses_with_position

    # make a dictionary of valid plays for one direction as a dictionary of words with a nested crossing dict
    # in the first entry of a list that are the values of the dictionary
    def valid_plays(self, valid_perms):
        # valid_perms = self.find_valid_perms() #TODO is this right to move to a parameter?

        valid_plays = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
        for valid_perm in valid_perms:
            potential_positions = valid_perms[valid_perm]
            for potential_position in potential_positions:
                y = potential_position[0]
                x = potential_position[1]
                crosses_valid, crosses_with_position = self.check_crosses(
                    valid_perm, y, x
                )
                if crosses_valid == True:
                    # TODO: I need to be able to add new positionf of the same cross here
                    # probably need to make another default dict here
                    for cross_with_position in crosses_with_position:
                        valid_plays[valid_perm][tuple(potential_position)][
                            cross_with_position[0]
                        ].append((cross_with_position[1:3]))
        return valid_plays

    # remember the crosses returned above have the x and y switched, as well as nested default dicts

    # Find which tiles of a word originated from the hand
    def new_tiles(self, valid_word, y, x):
        new_tiles = []
        for letter in valid_word:
            if self.check_yx(y, x) == False:
                new_tiles.append(
                    [letter, y, x]
                )  # should this really be a dictionary list?
            x = x + 1
        return new_tiles

    # Find which tiles could be used to build a crossing word
    def find_edge_hooks(self, valid_plays):
        edge_hooks = []
        for valid_word in valid_plays:
            for position in valid_plays[valid_word]:
                y = position[0]
                x = position[1]
                new_tiles = self.new_tiles(valid_word, y, x)
                if len(new_tiles) == 1:
                    edge_hooks = edge_hooks + new_tiles
        return edge_hooks

    # create a dictionary of the valid edge words
    # could probably adapt find valid perms above for this
    def valid_edge_words(self, edge_hooks):
        edge_perms_dict = {}
        for hook in edge_hooks:
            hook_letter = hook[0]
            # x and y are transposed
            y = hook[2]
            x = hook[1]
            edge_perms = self.edge_perms(hook_letter, y, x)

            for complete_word in edge_perms:
                if complete_word not in edge_perms_dict:
                    edge_perms_dict[complete_word] = edge_perms[complete_word]
                elif edge_perms[complete_word][0] not in edge_perms_dict[complete_word]:
                    edge_perms_dict[complete_word] = (
                        edge_perms_dict[complete_word] + edge_perms[complete_word]
                    )
        # copied from above, checking if perm in the dictionary
        edge_perms_set = set(edge_perms_dict.keys())
        valid_edge_perms = self.WORDSET.intersection(edge_perms_set)
        valid_edge_perms_dict = {
            k: v for (k, v) in edge_perms_dict.items() if k in valid_edge_perms
        }
        return valid_edge_perms_dict

    # Flattens the row and column dictionaries, making them more intuitive/combinable
    def unpack_dictionary(self, dict1):
        new_dict = {}
        for word in dict1:
            location_crosses = dict1[word]
            for location in location_crosses:
                crosses_dict = location_crosses[location]
                last_word_location = []
                for crossing_word in crosses_dict:
                    crossing_word_locations = crosses_dict[crossing_word]
                    for crossing_location in crossing_word_locations:
                        if len(last_word_location) > 0:
                            new_dict[(word, location)] = new_dict[
                                last_word_location
                            ] + (crossing_word, crossing_location)
                        else:
                            new_dict[(word, location)] = (
                                crossing_word,
                                crossing_location,
                            )
                        last_word_location = (word, location)
        new_new_dict = {}
        for key in new_dict:
            crossers_list = new_dict[key]
            new_new_dict[key] = tuple(zip(crossers_list, crossers_list[1:]))[::2]
        return new_new_dict

    # Returns 2 dictionaries of all the valid words and their crosses
    def all_board_words(self):

        # find the valid row plays
        valid_row_perms = self.find_valid_perms()

        valid_row_plays = self.valid_plays(valid_row_perms)

        # find the edge hooks for the column-wise edge words
        edge_hooks_for_columns = self.find_edge_hooks(valid_row_plays)

        # transpose
        self.board = self.board.transpose()

        # find the valid column plays
        valid_column_perms = self.find_valid_perms()
        valid_column_plays = self.valid_plays(valid_column_perms)

        # find the valid column edge plays
        valid_column_edge_perms = self.valid_edge_words(edge_hooks_for_columns)
        valid_column_edge_plays = self.valid_plays(valid_column_edge_perms)

        # find the edge hooks for the row-wise edge words
        edge_hooks_for_rows = self.find_edge_hooks(valid_column_plays)

        # transpose
        self.board = self.board.transpose()

        # find the valid row edge plays
        valid_row_edge_perms = self.valid_edge_words(edge_hooks_for_rows)
        valid_row_edge_plays = self.valid_plays(valid_row_edge_perms)

        # transform them all into a more readable format
        valid_row_plays = self.unpack_dictionary(valid_row_plays)
        valid_row_edge_plays = self.unpack_dictionary(valid_row_edge_plays)
        valid_column_plays = self.unpack_dictionary(valid_column_plays)
        valid_column_edge_plays = self.unpack_dictionary(valid_column_edge_plays)

        # collapse the row and column dictionaries
        all_row_words = {**valid_row_plays, **valid_row_edge_plays}
        all_column_words = {**valid_column_plays, **valid_column_edge_plays}

        # now I have two dictionaries: row and column in the format of:
        # {(mainword, y, x): ((crossingword, y, x), (crossingword, y, x), etc)
        # (mainword, y, x): ((crossingword, y, x), etc)
        # ...
        # }
        # remember that the y and x of the crossing words refer to a transposed board vis-a-vis the main word
        # Also, the crossing words' x and y will be referring to the transposed board, so they are switched

        return all_row_words, all_column_words

    # calculate the score of a group of tiles (not the whole play)
    def calculate_word_score(
        self, word, y, x, letter_multipliers_array, word_multipliers_array
    ):
        letter_multipliers = np.copy(letter_multipliers_array[y, x : x + len(word)])
        word_multipliers = np.copy(word_multipliers_array[y, x : x + len(word)])
        scores = [self.TILE_SCORES[l] for l in word]
        word_score = np.dot(letter_multipliers, scores)
        word_score = word_score * np.prod(word_multipliers)
        return word_score

    # calculating the total value of a main word and its crosses
    def total_score(self, main_word_and_location, crossing_words_and_locations, adjustion=False):
        letter_multipliers_array = np.copy(self.LETTER_MULTIPLIERS_ARRAY)
        word_multipliers_array = np.copy(self.WORD_MULTIPLIERS_ARRAY)

        # make the  multipliers==1 if there are pieces on them on the board
        # TODO: how permanent is the below?
        bool_mask = (self.board != "").astype(int)
        letter_multipliers_array = np.where(bool_mask == 1, 1, letter_multipliers_array)
        word_multipliers_array = np.where(bool_mask == 1, 1, word_multipliers_array)

        main_word = main_word_and_location[0]
        y = main_word_and_location[1][0]
        x = main_word_and_location[1][1]
        m_y, m_x = y, x
        main_score = self.calculate_word_score(
            main_word, y, x, letter_multipliers_array, word_multipliers_array
        )

        # transpose the multiplier arrays to use to score the crossing words
        T_word_multipliers = np.copy(word_multipliers_array.transpose())
        T_letter_multipliers = np.copy(letter_multipliers_array.transpose())

        # calculating the crossing words' total score
        crossers_score = 0
        for crossing_word_and_location in crossing_words_and_locations:
            if len(crossing_word_and_location[0]) > 1:
                y = crossing_word_and_location[1][0]
                x = crossing_word_and_location[1][1]
                crossing_word = crossing_word_and_location[0]
                crosser_score = self.calculate_word_score(
                    crossing_word, y, x, T_letter_multipliers, T_word_multipliers
                )
            else:
                crosser_score = 0
            crossers_score = crossers_score + crosser_score
        placed_tiles=self.new_tiles(main_word, m_y, m_x)
        total_score = (
            main_score
            + crossers_score
            + int(len(placed_tiles) == 7) * 35
        )
        
        #allow people to use 'adjusted values', which takes into account the future value of remaining hand tiles
        if adjustion=='leave_values':
            hand_letters = self.hand_letters.copy()
            LEAVE_VALUES_DICT=self.LEAVE_VALUES
            for letter in placed_tiles:
                hand_letters.remove(letter[0])
            hand_value=sum([LEAVE_VALUES_DICT[tile] for tile in hand_letters])
            total_score=total_score+hand_value
        return total_score

    # Create a dictionary of all valid main words and their scores
    def valid_score_choices(self, adjustion=False):
        board = self.board
        hand_letters = self.hand_letters
        letter_multiplier_array = self.LETTER_MULTIPLIERS_ARRAY
        word_multiplier_array = self.WORD_MULTIPLIERS_ARRAY

        # make a transpose board
        transpose_board = np.copy(board)
        transpose_board = transpose_board.transpose()

        # calculate the words/placement allowed by the rules of the game
        all_row_words, all_column_words = self.all_board_words()

        # create a dictionary of each of these row words along with their score
        row_score_dict = {}
        column_score_dict = {}
        for main_word_and_location in all_row_words:
            crossing_words_and_locations = all_row_words[main_word_and_location]
            score = self.total_score(
                main_word_and_location, crossing_words_and_locations, adjustion=adjustion #TODO: is this BP?
            )
            row_score_dict[main_word_and_location] = score

        # transpose the board
        self.board = self.board.transpose()

        # same for the column words
        for main_word_and_location in all_column_words:
            crossing_words_and_locations = all_column_words[main_word_and_location]
            score = self.total_score(
                main_word_and_location, crossing_words_and_locations, adjustion=adjustion
            )
            column_score_dict[main_word_and_location] = score

        self.board = self.board.transpose()
        self.row_mainword_scores = row_score_dict
        self.column_mainword_scores = column_score_dict
        return row_score_dict, column_score_dict

    # Return the highest scoring k words for rows and columns, if you have already run valid_score_choices
    def top_k_words(self, k):
        top_k_rows = {
            k: v
            for k, v in sorted(
                self.row_mainword_scores.items(), key=lambda item: item[1], reverse=True
            )[:k]
        }
        top_k_columns = {
            k: v
            for k, v in sorted(
                self.column_mainword_scores.items(),
                key=lambda item: item[1],
                reverse=True,
            )[:k]
        }
        return top_k_rows, top_k_columns

In [20]:
sample_board = [
    ["", "", "", "", "", "", "", "", "", "", "d", "e", "a", "d", ""],  # 0
    ["", "", "", "", "", "", "", "", "", "z", "o", "n", "a", "", "w"],  # 1
    ["", "", "", "", "", "", "", "", "", "", "", "a", "", "", "i"],  # 2
    ["", "", "", "", "", "", "", "", "", "r", "i", "m", "a", "", "l"],  # 3
    ["", "", "", "", "", "", "", "", "", "", "", "o", "b", "e", "y"],  # 4
    ["", "", "", "", "", "", "", "", "", "j", "u", "r", "a", "l", ""],
    ["", "", "", "", "", "", "", "b", "e", "e", "p", "", "", "", ""],  # 6
    ["", "", "", "f", "", "", "y", "e", "w", "s", "", "t", "", "", ""],
    ["", "", "", "o", "", "", "", "l", "e", "t", "c", "h", "", "", ""],  # 8
    ["", "", "", "c", "h", "u", "r", "l", "", "", "", "i", "", "r", ""],
    ["", "", "g", "i", "", "", "", "e", "", "", "", "g", "", "e", ""],  # 10
    ["", "", "i", "", "", "", "", "n", "", "", "", "h", "u", "e", "d"],
    ["", "", "n", "", "", "", "", "d", "", "", "", "", "", "f", ""],  # 12
    ["", "", "", "", "", "", "", "", "", "v", "e", "e", "p", "s", ""],
    ["", "", "", "", "", "", "", "", "n", "u", "n", "", "", "", ""],  # 14
]
sample_board_letters = ["o", "a", "s", "r", "l",'r','t']

# numpy of the board
board_array = np.array(sample_board)

# making the empty strings spaces
neat_sample_array = np.copy(board_array)
neat_sample_array[neat_sample_array == ""] = " "


In [21]:
print(np.where(board_array == "", " ", board_array))

[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'd' 'e' 'a' 'd' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'z' 'o' 'n' 'a' ' ' 'w']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'a' ' ' ' ' 'i']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'r' 'i' 'm' 'a' ' ' 'l']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'o' 'b' 'e' 'y']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'j' 'u' 'r' 'a' 'l' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' 'b' 'e' 'e' 'p' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' 'f' ' ' ' ' 'y' 'e' 'w' 's' ' ' 't' ' ' ' ' ' ']
 [' ' ' ' ' ' 'o' ' ' ' ' ' ' 'l' 'e' 't' 'c' 'h' ' ' ' ' ' ']
 [' ' ' ' ' ' 'c' 'h' 'u' 'r' 'l' ' ' ' ' ' ' 'i' ' ' 'r' ' ']
 [' ' ' ' 'g' 'i' ' ' ' ' ' ' 'e' ' ' ' ' ' ' 'g' ' ' 'e' ' ']
 [' ' ' ' 'i' ' ' ' ' ' ' ' ' 'n' ' ' ' ' ' ' 'h' 'u' 'e' 'd']
 [' ' ' ' 'n' ' ' ' ' ' ' ' ' 'd' ' ' ' ' ' ' ' ' ' ' 'f' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'v' 'e' 'e' 'p' 's' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'n' 'u' 'n' ' ' ' ' ' ' ' ']]


In [22]:
sam = Scrabbler(board_array, sample_board_letters)
sam.valid_score_choices(adjustion='leave_values')

({('ta', (7, 11)): 9.2,
  ('tao', (7, 11)): 10.0,
  ('tao', (8, 1)): 10.7,
  ('tas', (7, 11)): 2.7,
  ('tar', (7, 11)): 9.7,
  ('stoa', (8, 1)): 10.2,
  ('tosa', (7, 11)): 3.5,
  ('tora', (7, 11)): 10.5,
  ('tola', (7, 11)): 13.0,
  ('tsar', (7, 11)): 3.2,
  ('of', (7, 2)): 13.7,
  ('to', (7, 11)): 10.7,
  ('to', (8, 2)): 11.4,
  ('fa', (7, 3)): 12.2,
  ('fa', (12, 13)): 15.2,
  ('oaf', (7, 1)): 13.0,
  ('orf', (7, 1)): 14.2,
  ('tor', (7, 11)): 11.2,
  ('oft', (7, 2)): 15.2,
  ('tot', (7, 11)): 12.2,
  ('arf', (7, 1)): 12.7,
  ('alf', (7, 1)): 15.2,
  ('aft', (7, 2)): 13.7,
  ('tat', (7, 11)): 10.7,
  ('oafs', (7, 1)): 6.5,
  ('tost', (7, 11)): 5.7,
  ('orfs', (7, 1)): 7.7,
  ('tors', (7, 11)): 4.7,
  ('torr', (7, 11)): 11.7,
  ('tort', (7, 11)): 12.7,
  ('tolt', (7, 11)): 15.2,
  ('tots', (7, 11)): 5.7,
  ('taos', (7, 11)): 3.5,
  ('taos', (8, 1)): 10.2,
  ('taro', (7, 11)): 10.5,
  ('taro', (8, 0)): 13.2,
  ('arfs', (7, 1)): 6.2,
  ('tars', (7, 11)): 3.2,
  ('tart', (7, 11)): 11.2,


In [18]:
#with the leave values adjustion
sam.top_k_words(5)

({('rostral', (13, 0)): 61,
  ('nuns', (14, 8)): 28.4,
  ('salto', (13, 2)): 23.0,
  ('slot', (13, 2)): 22.7,
  ('slart', (13, 2)): 22.7},
 {('ewest', (8, 6)): 26.9,
  ('alt', (3, 12)): 23.7,
  ('holt', (4, 9)): 23.2,
  ('lo', (8, 10)): 22.7,
  ('rho', (4, 8)): 22.2})

In [14]:
#without the leave values adjustion
#seems like LV is just adding 10 points to everything
#TODO: fix that 
sam.top_k_words(5)

({('rostral', (13, 0)): 61,
  ('nuns', (14, 8)): 27,
  ('slart', (13, 2)): 22,
  ('soral', (13, 2)): 22,
  ('solar', (13, 2)): 22},
 {('ewest', (8, 6)): 25,
  ('ewes', (8, 6)): 20,
  ('shorl', (4, 8)): 19,
  ('shoal', (4, 8)): 19,
  ('lash', (4, 6)): 18})

**More accurate (Scrabble) leave values:**

Subtract .5 from the consonants and add .7 to the vowels for the individual approximate leave values:

S 8.04
Z 5.12
X 3.31
R 1.10
H 1.09
C 0.85
M 0.58
D 0.45
E 0.35
N 0.22
T -0.10
L -0.17
P -0.46
K -0.54
Y -0.63
A -0.63
J -1.47
B -2.00
I -2.07
F -2.21
O -2.50
G -2.85
W -3.82
U -5.10
V -5.55
Q -6.79